In [1]:
!pip install onnxruntime-gpu
!pip install onnxconverter-common
!pip install apache-tvm
!pip install torch
!pip install numpy
!pip install onnxruntime
!pip install onnx
!pip install timm
!pip install tqdm
!pip install torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requ

In [2]:
# import libraries
import os
import torch
import json
import time
from tqdm.auto import tqdm
import timm
import numpy as np
import onnxruntime as ort
from onnxruntime import quantization
from onnxconverter_common import float16
import onnx
import tvm
from tvm import relay



In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device = ", device)


device =  cuda


In [4]:
def preprocess(model):
    data_config = timm.data.resolve_model_data_config(model)
    transforms = timm.data.create_transform(**data_config, is_training = False)
    print(transforms)
    
    val_dataset = timm.data.ImageDataset('/kaggle/input/imagenet/imagenet-mini', transform = transforms)
    val_loader = timm.data.create_loader(val_dataset, (1, 3, 224, 224), 1)
    
    val_dataset_sub = torch.utils.data.Subset(val_dataset, list(range(1000)))
    val_loader_sub = timm.data.create_loader(val_dataset_sub, (1, 3, 224, 224), 1)
    
    return val_loader, val_loader_sub

print("Preprocess Done ...!")

Preprocess Done ...!


In [5]:
class OnnxStaticQuantization:
    def __init__(self) -> None:
        self.enum_data = None
        self.calibration_technique = {
            "MinMax": ort.quantization.calibrate.CalibrationMethod.MinMax,
            "Entropy": ort.quantization.calibrate.CalibrationMethod.Entropy,
            "Percentile": ort.quantization.calibrate.CalibrationMethod.Percentile,
            "Distribution": ort.quantization.calibrate.CalibrationMethod.Distribution
        }

    def get_next(self, EP_list = ['CPUExecutionProvider']):
        if self.enum_data is None:
            session = ort.InferenceSession(self.fp32_onnx_path, providers = EP_list)
            input_name = session.get_inputs()[0].name
            calib_list = []
            count = 0
            for nhwc_data, _ in self.calibration_loader:
                nhwc_data=nhwc_data.cpu()
                calib_list.append({input_name: nhwc_data.numpy()}) 
                if self.sample == count: break
                count = count + 1
            self.enum_data = iter(calib_list)
        return next(self.enum_data, None)
    
    def quantization(self, fp32_onnx_path, future_int8_onnx_path, calib_method, calibration_loader, sample = 100):
        self.sample = sample
        self.calibration_loader = calibration_loader 
        _ = ort.quantization.quantize_static(
                model_input = fp32_onnx_path,
                model_output = future_int8_onnx_path,
                calibrate_method = self.calibration_technique[calib_method],
                activation_type=ort.quantization.QuantType.QInt16,
                weight_type=ort.quantization.QuantType.QInt8,
                per_channel = True, 
                reduce_range = True,
                calibration_data_reader = self
            )
        return self


print("Quantization Done ...!")

Quantization Done ...!


In [6]:
# Quantization Investigation
def quant_investigation(quant_model_name):
    _model = onnx.load(quant_model_name + ".onnx")
    initializers = _model.graph.initializer

    for node_i in _model.graph.node:
        if node_i.output and "QuantizeLinear" not in node_i.output[0] and "DequantizeLinear" not in node_i.name:
            for node_j in _model.graph.node:
                if node_j.input and node_i.output[0] == node_j.input[0] and "QuantizeLinear" not in node_j.output[0]:
                    print(node_i.name)

print("Quantization Investigation Done ...!")


Quantization Investigation Done ...!


In [7]:
def validate(model, val_loader, model_name, ONNX = False, quant = "", sample_size = 100, quant_invest = False, TVM = False):
    correct = 0
    total = 0
    elapsed_time = 0
    top5_correct = 0
    
    if ONNX:    # 1
        if not TVM:    # 2  
            if quant == "fp16":    # 3
                model = onnx.load(model_name + ".onnx")
                model_fp16 = float16.convert_float_to_float16(model)
                onnx.save(model_fp16,model_name + quant + ".onnx")
                
            elif quant == "int8":    # 3
                ort.quantization.shape_inference.quant_pre_process(model_name + ".onnx", "preprocess.onnx")
                module = OnnxStaticQuantization()
                module.fp32_onnx_path = "preprocess.onnx"
                
                module.quantization(
                    fp32_onnx_path = "preprocess.onnx",
                    future_int8_onnx_path = model_name + quant + ".onnx",
#                     calib_method = "MinMax",
#                     calib_method = "Entropy",
                    calib_method = "Percentile",
#                     calib_method = "Distribution",
                    calibration_loader = val_loader,
                    sample = sample_size
                )
                
            else:    # 3
                dummy_input = torch.randn(1, 3, 224, 224).to(device)
                torch.onnx.export(model, dummy_input, model_name + ".onnx", export_params = True, opset_version = 14, do_constant_folding = True)
                
                
        else:    # 2
            onnx_model = onnx.load(model_name + quant + ".onnx")
            shape_dict = {"input.1": (1, 3, 224, 224)}
            mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)
            target = "llvm -mcpu=core-avx2"
            with tvm.transform.PassContext(opt_level = 2):
                executor = relay.build_module.create_executor("graph", mod, tvm.cpu(0), target, params).evaluate()
        
        # -------------------------------------------------------------------------------------------------------------------------
        
        sess = ort.InferenceSession(model_name + quant + ".onnx", providers = ['CPUExecutionProvider'])
        for batch_idx, (inputs, labels) in enumerate(tqdm(val_loader)):
            if quant == "fp16":
                inputs = inputs.half()
            inputs = inputs.cpu().numpy()
            
            if TVM:
                inputs = tvm.nd.array(inputs.astype("float32"))
                start_time = time.time()    
                outputs = [executor(inputs).numpy()]
                end_time = time.time()
                
            else:
                start_time = time.time()    
                outputs = sess.run(None, {sess.get_inputs()[0].name: inputs})
                end_time = time.time()
            
            elapsed_time += end_time - start_time

            predicted_labels = np.argmax(outputs[0], axis=1)

            correct += (predicted_labels == labels.cpu().numpy()).sum()
            total += labels.size(0)

            top5_predicted = np.argsort(outputs[0], axis=1)[:, -5:]  
            top5_correct += np.sum(np.equal(top5_predicted, np.expand_dims(labels.cpu().numpy(), axis=1)))
    
    
    elif ONNX == False and quant_invest == True and quant != "":    # 1
        if quant == "fp16" :
            print("fp16 quantization investigation : ")
            quant_investigation(model_name + quant)
            print()
        elif quant == "int8" :
            print("int8 quantization investigation : ")
            quant_investigation(model_name + quant)
            print()
        else :
            print("Error : quant is empty.")
        return
    
    else:  # 1
        model = model.eval()
        with torch.inference_mode():
            for batch_idx, (images, labels) in enumerate(tqdm(val_loader)):
                images, labels = images.to(device), labels.to(device)
                
                start_time = time.time()
                outputs = model(images)
                end_time = time.time()
                elapsed_time += end_time - start_time
                
                _, predicted = torch.max(outputs.softmax(dim = 1) * 100, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                _, top5_predicted = torch.topk(outputs.softmax(dim = 1) * 100, k = 5)
                top5_correct += sum(labels[i].item() in top5_predicted[i] for i in range(len(labels)))
    
    # =============================================================================================================================

    # Calculate accuracy
    single_inference_runtime = elapsed_time / total
    top1Accuracy = correct / total
    top5Accuracy = top5_correct / total
    
    print('Single Inference Runtime: {:.4f} seconds\nTop 1 Accuracy : {:.2f}%\nTop 5 Accuracy : {:.2f}%\n'.format(single_inference_runtime, 100 * top1Accuracy, 100 * top5Accuracy))


print("Validation Done ...!")

Validation Done ...!


In [8]:
# import torchvision.models as tm
# model = tm.maxvit_t('vit_base_patch16_384', pretrained=True).to(device)
# from torchvision.models import MaxVit_T_Weights
# model = timm.create_model('vit_base_patch16_384', pretrained=True).to(device)

# model load
model = timm.create_model('vit_base_patch16_224', pretrained=True).to(device)
torch.save(model.state_dict(), 'vit_base_patch16_224.pth')
print("Model Saved ...!")

val_loader, val_loader_sub = preprocess(model)
print("Data Preprocessing Done.")


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Model Saved ...!
Compose(
    Resize(size=248, interpolation=bicubic, max_size=None, antialias=warn)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=tensor([0.5000, 0.5000, 0.5000]), std=tensor([0.5000, 0.5000, 0.5000]))
)
Data Preprocessing Done.


In [9]:
# Baseline Setup
validate(model, val_loader_sub, 'vit_base_patch16_224')


  0%|          | 0/1000 [00:00<?, ?it/s]

Single Inference Runtime: 0.0059 seconds
Top 1 Accuracy : 94.90%
Top 5 Accuracy : 99.70%



In [10]:
# Onnx Validation
validate(model, val_loader_sub, 'vit_base_patch16_224', ONNX=True)


/opt/conda/lib/python3.10/site-packages/torch/__init__.py:1404: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert condition, message


  0%|          | 0/1000 [00:00<?, ?it/s]

Single Inference Runtime: 0.2128 seconds
Top 1 Accuracy : 94.90%
Top 5 Accuracy : 99.70%



In [11]:
# Model Quantization fp16
validate(model, val_loader_sub, 'vit_base_patch16_224', ONNX=True, quant="fp16")


/opt/conda/lib/python3.10/site-packages/onnxconverter_common/float16.py:43: UserWarning: the float32 number 4.487844051936918e-08 will be truncated to 1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(pos_min, min_positive_val))
/opt/conda/lib/python3.10/site-packages/onnxconverter_common/float16.py:53: UserWarning: the float32 number -3.018414318489704e-08 will be truncated to -1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(neg_max, -min_positive_val))
/opt/conda/lib/python3.10/site-packages/onnxconverter_common/float16.py:43: UserWarning: the float32 number 8.422011887887493e-08 will be truncated to 1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(pos_min, min_positive_val))
/opt/conda/lib/python3.10/site-packages/onnxconverter_common/float16.py:43: UserWarning: the float32 number 1.5813768072803214e-09 will be truncated to 1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(

  0%|          | 0/1000 [00:00<?, ?it/s]

Single Inference Runtime: 0.4523 seconds
Top 1 Accuracy : 94.90%
Top 5 Accuracy : 99.70%



In [12]:
# Model QDQ Investigation
validate(model, val_loader_sub, 'vit_base_patch16_224', ONNX=False, quant="fp16", quant_invest=True)


fp16 quantization investigation : 
/patch_embed/proj/Conv
/patch_embed/proj/Conv
/patch_embed/Shape
/patch_embed/Slice
/patch_embed/Reshape
/Constant_1
/Constant_2
/ConstantOfShape
/Equal
/Expand
/Concat
/Add
/Add
/Add
/blocks/blocks.0/norm1/Sub
/blocks/blocks.0/norm1/Sub
/blocks/blocks.0/norm1/Pow
/blocks/blocks.0/norm1/ReduceMean_1
/blocks/blocks.0/norm1/Add
/blocks/blocks.0/norm1/Div
/blocks/blocks.0/norm1/Mul
/blocks/blocks.0/norm1/Add_1
/blocks/blocks.0/attn/qkv/Add
/blocks/blocks.0/attn/Reshape
/blocks/blocks.0/attn/Transpose
/blocks/blocks.0/attn/Split
/blocks/blocks.0/attn/Squeeze
/blocks/blocks.0/attn/Squeeze
/blocks/blocks.0/attn/Squeeze_1
/blocks/blocks.0/attn/Shape
/blocks/blocks.0/attn/Slice
/blocks/blocks.0/attn/Cast
/blocks/blocks.0/attn/Constant_7
/blocks/blocks.0/attn/Div
/blocks/blocks.0/attn/Div
/blocks/blocks.0/attn/Transpose_1
/blocks/blocks.0/attn/Mul
/blocks/blocks.0/attn/MatMul
/blocks/blocks.0/attn/Softmax
/blocks/blocks.0/attn/MatMul_1
/blocks/blocks.0/attn/Tr

In [13]:
# Model Quantization int8
validate(model, val_loader_sub, 'vit_base_patch16_224', ONNX=True, sample_size=50, quant="int8")


Finding optimal threshold for each tensor using 'percentile' algorithm ...
Number of tensors : 505
Number of histogram bins : 2048
Percentile : (0.0010000000000047748,99.999)


/opt/conda/lib/python3.10/site-packages/onnxruntime/quantization/onnx_quantizer.py:919: RuntimeWarning: invalid value encountered in cast
  quantized_data = (np.asarray(bias_data) / bias_scale).round().astype(np.int32)


  0%|          | 0/1000 [00:00<?, ?it/s]

Single Inference Runtime: 0.5787 seconds
Top 1 Accuracy : 87.00%
Top 5 Accuracy : 96.60%



In [14]:
# Model QDQ Investigation
validate(model, val_loader_sub, 'vit_base_patch16_224', ONNX=False, quant="int8", quant_invest=True)


int8 quantization investigation : 
/blocks/blocks.0/norm1/Sub
/blocks/blocks.0/norm1/Sub
/blocks/blocks.0/norm1/Pow
/blocks/blocks.0/norm2/Sub
/blocks/blocks.0/norm2/Sub
/blocks/blocks.0/norm2/Pow
/blocks/blocks.0/mlp/act/Div
/blocks/blocks.1/norm1/Sub
/blocks/blocks.1/norm1/Sub
/blocks/blocks.1/norm1/Pow
/blocks/blocks.1/norm2/Sub
/blocks/blocks.1/norm2/Sub
/blocks/blocks.1/norm2/Pow
/blocks/blocks.1/mlp/act/Div
/blocks/blocks.2/norm1/Sub
/blocks/blocks.2/norm1/Sub
/blocks/blocks.2/norm1/Pow
/blocks/blocks.2/norm2/Sub
/blocks/blocks.2/norm2/Sub
/blocks/blocks.2/norm2/Pow
/blocks/blocks.2/mlp/act/Div
/blocks/blocks.3/norm1/Sub
/blocks/blocks.3/norm1/Sub
/blocks/blocks.3/norm1/Pow
/blocks/blocks.3/norm2/Sub
/blocks/blocks.3/norm2/Sub
/blocks/blocks.3/norm2/Pow
/blocks/blocks.3/mlp/act/Div
/blocks/blocks.4/norm1/Sub
/blocks/blocks.4/norm1/Sub
/blocks/blocks.4/norm1/Pow
/blocks/blocks.4/norm2/Sub
/blocks/blocks.4/norm2/Sub
/blocks/blocks.4/norm2/Pow
/blocks/blocks.4/mlp/act/Div
/blocks/bl

In [15]:
# Quantization Investigation
import onnx

def quant_investigation(quant_model_name):
    _model = onnx.load(quant_model_name + ".onnx")
    initializers = _model.graph.initializer

    for node_i in _model.graph.node:
        if node_i.output and "QuantizeLinear" not in node_i.output[0] and "DequantizeLinear" not in node_i.name:
            for node_j in _model.graph.node:
                if node_j.input and node_i.output[0] == node_j.input[0] and "QuantizeLinear" not in node_j.output[0]:
                    print(node_i.name)

print("fp16 quantization investigation : ")
quant_investigation("vit_base_patch16_224fp16")
print()


fp16 quantization investigation : 
/patch_embed/proj/Conv
/patch_embed/proj/Conv
/patch_embed/Shape
/patch_embed/Slice
/patch_embed/Reshape
/Constant_1
/Constant_2
/ConstantOfShape
/Equal
/Expand
/Concat
/Add
/Add
/Add
/blocks/blocks.0/norm1/Sub
/blocks/blocks.0/norm1/Sub
/blocks/blocks.0/norm1/Pow
/blocks/blocks.0/norm1/ReduceMean_1
/blocks/blocks.0/norm1/Add
/blocks/blocks.0/norm1/Div
/blocks/blocks.0/norm1/Mul
/blocks/blocks.0/norm1/Add_1
/blocks/blocks.0/attn/qkv/Add
/blocks/blocks.0/attn/Reshape
/blocks/blocks.0/attn/Transpose
/blocks/blocks.0/attn/Split
/blocks/blocks.0/attn/Squeeze
/blocks/blocks.0/attn/Squeeze
/blocks/blocks.0/attn/Squeeze_1
/blocks/blocks.0/attn/Shape
/blocks/blocks.0/attn/Slice
/blocks/blocks.0/attn/Cast
/blocks/blocks.0/attn/Constant_7
/blocks/blocks.0/attn/Div
/blocks/blocks.0/attn/Div
/blocks/blocks.0/attn/Transpose_1
/blocks/blocks.0/attn/Mul
/blocks/blocks.0/attn/MatMul
/blocks/blocks.0/attn/Softmax
/blocks/blocks.0/attn/MatMul_1
/blocks/blocks.0/attn/Tr

In [16]:
# Model TVM with quant int8
validate(model, val_loader_sub, 'vit_base_patch16_224', ONNX = True, sample_size = 50, quant = 'int8', TVM = True)


  0%|          | 0/1000 [00:00<?, ?it/s]

Single Inference Runtime: 2.2313 seconds
Top 1 Accuracy : 86.90%
Top 5 Accuracy : 96.60%



In [17]:
# Model TVM with fp16
validate(model, val_loader_sub, 'vit_base_patch16_224', ONNX = True, sample_size = 50, quant = 'fp16', TVM = True)


2024-04-06 14:13:23.062272959 [W:onnxruntime:, constant_folding.cc:269 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Sqrt node '/blocks/blocks.0/attn/Sqrt'
2024-04-06 14:13:23.062549787 [W:onnxruntime:, constant_folding.cc:269 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Sqrt node '/blocks/blocks.1/attn/Sqrt'
2024-04-06 14:13:23.062802720 [W:onnxruntime:, constant_folding.cc:269 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Sqrt node '/blocks/blocks.2/attn/Sqrt'
2024-04-06 14:13:23.063042316 [W:onnxruntime:, constant_folding.cc:269 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Sqrt node '/blocks/blocks.3/attn/Sqrt'
2024-04-06 14:13:23.063286738 [W:onnxruntime:, constant_folding.cc:269 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Sqrt node '/blocks/blocks.4/attn/Sqrt'
2024-04-06 14:13:23.063523432 [W:onnxruntime:, constant_folding.cc:269 ApplyImpl] Could not find a CPU ke

  0%|          | 0/1000 [00:00<?, ?it/s]

TVMError: Traceback (most recent call last):
  191: 0x00005c68b85dfd10
  190: __libc_start_main
  189: Py_BytesMain
        at /usr/local/src/conda/python-3.10.13/Modules/main.c:1090
  188: Py_RunMain
        at /usr/local/src/conda/python-3.10.13/Modules/main.c:670
  187: pymain_run_python
        at /usr/local/src/conda/python-3.10.13/Modules/main.c:585
  186: pymain_run_module
        at /usr/local/src/conda/python-3.10.13/Modules/main.c:297
  185: _PyFunction_Vectorcall
        at /usr/local/src/conda/python-3.10.13/Objects/call.c:342
  184: _PyEval_Vector
        at /usr/local/src/conda/python-3.10.13/Python/ceval.c:5067
  183: _PyEval_EvalFrame
        at /usr/local/src/conda/python-3.10.13/Include/internal/pycore_ceval.h:46
  182: _PyEval_EvalFrameDefault
        at /usr/local/src/conda/python-3.10.13/Python/ceval.c:4213
  181: call_function
        at /usr/local/src/conda/python-3.10.13/Python/ceval.c:5893
  180: PyObject_Vectorcall
        at /usr/local/src/conda/python-3.10.1

In [ ]:
# import torch
# import torchvision.models as models
# import urllib
# from PIL import Image
# from torchvision import transforms

# # Create an instance of the MaxViT-T model with pre-trained weights on ImageNet
# model = models.maxvit_t(weights='imagenet')

# # Define a preprocessing pipeline for the input image
# preprocess = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

# # Load and preprocess an example image
# url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
# urllib.request.urlretrieve(url, filename)
# input_image = Image.open(filename)
# input_tensor = preprocess(input_image)
# input_batch = input_tensor.unsqueeze(0)  # Add a batch dimension

# # Put the model in evaluation mode
# model.eval()

# # Perform inference using the pre-trained model
# with torch.no_grad():
#     output = model(input_batch)

# # Print the top 5 predicted classes
# _, indices = torch.sort(output, descending=True)
# imagenet_labels = urllib.request.urlopen("https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json")
# labels = [line.strip() for line in imagenet_labels.readlines()]
# percentage = torch.nn.functional.softmax(output, dim=1)[0] * 100
# for i in range(5):
#     print(labels[indices[0][i]], percentage[indices[0][i]].item())
